In [61]:
import cv2
import numpy as np
import glob


In [ ]:
def detect_robot(image):

    height, width, _ = image.shape
    lower_quarter = image[4 * height // 5 :, :] #crop to the lower quarter (to ignore ceiling lights)

    # Convert to grayscale
    gray = cv2.cvtColor(lower_quarter, cv2.COLOR_BGR2GRAY)

    # Enhance brightness (of the robot light)
    bright = cv2.convertScaleAbs(gray, alpha=1.5, beta=0)

    # brightness thresholding
    brightness_threshold = 40 #TODO better threshold?
    _, mask = cv2.threshold(bright, brightness_threshold, 255, cv2.THRESH_BINARY)

    mask = cv2.bitwise_not(mask) #reversed mask to detect dark



    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on the lower half of the image
    #cv2.drawContours(lower_half, contours, -1, (0, 255, 0), 2)  # Green contours
    #print(len(contours))


    # Filter contours by length
    robot_detected = False
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > width // 9:  # Line is larger than 1/4 of the image width at which point we should dodge the robot TODO better strategy?
            robot_detected = True
            # Show detected line
            cv2.rectangle(lower_quarter, (x, y), (x + w, y + h), (0, 0, 255), 2)


    # Combine the lower quarter back with the upper half for display
    output_image = np.vstack((image[: height // 5 * 4, :], lower_quarter))

    return output_image, robot_detected, mask


In [63]:
#image = cv2.imread('images_obstacle/image_20241125_145857.jpg')
image = cv2.imread('images_obstacle/image_20241125_145833.jpg')
img_detected, robot_detected, img_mask = detect_robot(image)
print('Robot in image: ', robot_detected)

cv2.imwrite('processed_image.jpg', img_detected)
cv2.imwrite('mask_image.jpg', img_mask)


Robot in image:  True


True

In [64]:
image_files = glob.glob("images_obstacle/*.jpg")

for i, filename in enumerate(image_files):
    print(filename)
    img = cv2.imread(filename)
    img_detected, robot_detected, img_mask = detect_robot(img)

    cv2.imwrite(f'{filename}_detection.jpg', img_detected)
    cv2.imwrite(f'{filename}_mask.jpg', img_mask)


images_obstacle/image_20241125_150017.jpg
images_obstacle/image_20241125_150014.jpg
images_obstacle/image_20241125_145916.jpg
images_obstacle/image_20241125_145926.jpg
images_obstacle/image_20241125_150053.jpg
images_obstacle/image_20241125_150051.jpg
images_obstacle/image_20241125_150055.jpg
images_obstacle/image_20241125_145937.jpg
images_obstacle/image_20241125_145923.jpg
images_obstacle/image_20241125_145857.jpg
images_obstacle/image_20241125_145936.jpg
images_obstacle/image_20241125_145950.jpg
images_obstacle/image_20241125_145833.jpg
images_obstacle/image_20241125_145947.jpg
images_obstacle/image_20241125_145943.jpg
images_obstacle/image_20241125_145942.jpg
images_obstacle/image_20241125_150034.jpg
